In [1]:
import seaborn as sns
import numpy as np
import pandas as pd


tips = sns.load_dataset('tips')

for i in tips.columns[2:6]:
    labels = np.sort(tips[i].unique())
    mytype = pd.api.types.CategoricalDtype(categories=labels)
    print(i, labels)
    tips[i] = tips[i].astype(mytype).cat.codes

sex ['Female' 'Male']
smoker ['No' 'Yes']
day ['Fri' 'Sat' 'Sun' 'Thur']
time ['Dinner' 'Lunch']


In [4]:
# total_bill, tip --> StandardScaler
from sklearn.preprocessing import StandardScaler

X = tips['total_bill'].values
bill_scaler = StandardScaler().fit(X[:,np.newaxis])
bill_u = tips['total_bill'].mean()
bill_s = tips['total_bill'].std(ddof=0)
tips['total_bill'] = bill_scaler.transform(X[:,np.newaxis])

X = tips['tip'].values
tip_scaler = StandardScaler().fit(X[:,np.newaxis])
tips['tip'] = tip_scaler.transform(X[:,np.newaxis])

print(tips)

     total_bill       tip  sex  smoker  day  time  size
0     -0.314711 -1.439947    1       1    3     1     2
1     -1.063235 -0.969205    0       1    3     1     3
2      0.137780  0.363356    0       1    3     1     3
3      0.438315  0.225754    0       1    3     1     2
4      0.540745  0.443020    1       1    3     1     4
..          ...       ...  ...     ...  ...   ...   ...
239    1.040511  2.115963    0       1    2     1     3
240    0.832275 -0.722971    1       0    2     1     2
241    0.324630 -0.722971    0       0    2     1     2
242   -0.221287 -0.904026    0       1    2     1     2
243   -0.113229  0.001247    1       1    0     1     2

[244 rows x 7 columns]


In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# 일요일 저녁, 여자 3명 --> 얼마의 매상을 올릴것인가.
'''
sex ['Female' 'Male']
smoker ['No' 'Yes']
day ['Fri' 'Sat' 'Sun' 'Thur']
time ['Dinner' 'Lunch']
'''

X = tips[['day','time','sex','size']].values
y = tips['total_bill'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)
m = LinearRegression().fit(X_train, y_train)
print('train score ', m.score(X_train, y_train))
print('test score ', m.score(X_test, y_test))

z = m.predict([[2,0,0,3]])
print('predict ', z, (z*bill_s)+bill_u)   # z = (x - u) / s

train score  0.3887307452205975
test score  0.3600931113979351
predict  [0.02443901] [20.00306248]


In [11]:
# 3명의 손님이 와서 팁으로 5.00를 받았다면, 매상은 얼마인것일까..

# indep_var : size, tip
# dep_var : total_bill

X = tips[['size', 'tip']].values
y = tips['total_bill'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

m = LinearRegression().fit(X_train, y_train)
print('train score ', m.score(X_train, y_train))
print('test score ', m.score(X_test, y_test))

tip_value = tip_scaler.transform([[5.00]])
print(tip_value[0,0])
z = m.predict([[3, tip_value[0,0]]])
print('predict ', z, bill_scaler.inverse_transform(z[:,np.newaxis]))

train score  0.5624715269527789
test score  0.5179720133891708
1.449682342415553
predict  [0.87643709] [[27.57234168]]
